In [1]:
import numpy as np
import sys

from utils import Node, Problem

In [2]:
def objectiv_func(W, R, lmd=2):
    return np.dot(W, R) - lmd * np.sqrt(np.dot(W, W) * np.cov(W))

In [3]:
def rand_wight(n=3):
    W = []
    low = 0
    high = 100
    for _ in range(n):
        rnd = np.random.randint(low, high)
        W.append(rnd)
        high -= rnd
    if np.sum(W) < 100:
        W[-1] += 100 - np.sum(W)
    return np.array(W, dtype=float)

## genetic

In [4]:
def one_point_crossover(parent1, parent2):
    p1, p2 = list(parent1), list(parent2)
    rand = np.random.choice(len(parent1))
    child1, child2 = (p1[:rand] + p2[rand:], p2[:rand] + p1[rand:])
    return  np.array(child1), np.array(child2)

def increase_decrease_crossover(parent1, parent2, rate: float=0.1):
    child1, child2 = [], []
    for x1, x2 in zip(list(parent1), list(parent2)):
        dif = x1 - x2
        if dif > 0:
            child1.append(x1 - rate * dif)
            child2.append(x2 + rate * dif)
        else:
            child1.append(x1 + rate * dif)
            child2.append(x2 - rate * dif)
    return  np.array(child1), np.array(child2)

def balanced_increase_decrease_crossover(parent1, parent2, rate: float=0.1):
    n = len(parent1)
    idx = np.random.choice(n)
    dif = parent1[idx] - parent2[idx]
    dif *= rate
    if dif > 0:
        parent1[idx] -= dif 
        parent2[idx] += dif 
        for i in range(n):
            if i != idx:
                parent1[i] += dif / (n-1)
                parent2[i] -= dif / (n-1)
    else:
        parent1[idx] += dif 
        parent2[idx] -= dif 
        for i in range(n):
            if i != idx:
                parent1[i] -= dif / (n-1)
                parent2[i] += dif / (n-1)
    return np.array(parent1), np.array(parent2)

In [5]:
def initial_pop(stock_num=3, n=100):
    return [rand_wight(stock_num) for _ in range(n)]

def fit_min(pop, R):
    fmin = min(pop, key=lambda x: objectiv_func(W=x, R=R))
    return objectiv_func(fmin, R)

def fitness(chorm, fmin, R):
    return objectiv_func(W=chorm, R=R) + np.abs(fmin)

def crossover(parent1, parent2):
    return balanced_increase_decrease_crossover(parent1, parent2)

def tournament_selection(pop, R, fmin, T):
    idx1, idx2 = tuple(np.random.choice(len(pop), 2, replace=False))
    p1, p2 = pop[idx1], pop[idx2]
    fi, fj = fitness(p1, fmin, R), fitness(p2, fmin, R)
    if np.random.rand() < 1 / (1 + np.exp(-(fi - fj) / T)):
        return p1
    else:
        return p2

In [6]:
def run(R, stock_num=3, num_iter=50):
    pop = initial_pop(stock_num)
    fmin = fit_min(pop, R)
    for t in range(1, num_iter):
        # selection
        mating_pool =  [tournament_selection(pop, R, fmin, t) for _ in range(len(pop))]
        # crossover
        for _ in range(len(mating_pool)//2):
            idx1 = np.random.choice(len(mating_pool), 1)
            p1 = mating_pool.pop(idx1[0])
            idx2 = np.random.choice(len(mating_pool), 1)
            p2 = mating_pool.pop(idx2[0])
            pop.extend(list(crossover(p1, p2)))
        # survive selection
        fmin = fit_min(pop, R)
        pop = sorted(pop, key=lambda x: fitness(x, fmin, R))
        pop = pop[100:]
 
    return pop[-1], fitness(pop[-1], fmin, R)

In [59]:
r = np.array([15, 10, 2])
solution, solution_fitness = run(R=r, stock_num=3, num_iter=200)
print(f"solution fitness: {solution_fitness}")
print(f"solution: {solution}")
print(f"objectiov function: {objectiv_func(solution, r)}")
# print(f"sum solution: {sum(solution)}")

/tmp/ipykernel_5292/3080108440.py:18: RuntimeWarning: overflow encountered in exp
  if np.random.rand() < 1 / (1 + np.exp(-(fi - fj) / T)):


solution fitness: 1799.6504360321526
solution: [33.33280242 33.33235551 33.33484207]
objectiov function: 899.8322116650011


## simulated annealing

In [8]:
class Portfolio(Problem):
    def __init__(self, initial, restitution, lamda=1 ,step=0.1):
        self.initial = initial
        self.restitution = restitution
        self.lamda = lamda
        self.step  = step

    def actions(self, state):
        """n**2 - n or O(n**2)"""
        acts = []
        for i in range(len(state)):
            for j in range(len(state)):
                if j != i:
                    acts.append({"up": i, "down": j})
        return acts
    
    def result(self, state, action):
        new_state = state.copy()
        new_state[action["up"]] += self.step
        new_state[action["down"]] -= self.step
        return new_state
    
    def value(self, state):
        def objectiv_func(W, R, lmd=1):
            return np.dot(W, R) - lmd * np.sqrt(np.dot(W, W) * np.cov(W))
        return objectiv_func(state, self.restitution, self.lamda)


In [23]:
## book pseudo code
def simulated_annealing(problem, schedul=lambda x: 1/(x+1), iter=1000):
    current = Node(problem.initial)
    for t in range(iter):
        T = schedul(t)
        if T == 0:
            return current
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = np.random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0:
            current = next_choice
        else:
            if np.random.rand() > delta_e:
                current = next_choice
    return current

In [57]:
w = np.array([25, 46, 29], dtype=float)
r = np.array([15, 10, 2], dtype=float)
max = 0
max_state = None
for i in range(10):
    p = Portfolio(initial=w, restitution=r, step=0.5)
    res = simulated_annealing(p)
    # print("iter:", i+1)
    if p.value(res.state) > max:
        max = p.value(res.state)
        max_state = res.state

print(f"max: {max}")
print(max_state)

max: 785.0711449885839
[35.5 34.  30.5]


In [11]:
def exp_schedule(k=20, lam=0.005, limit=100):
    """One possible schedule function for simulated annealing"""
    return lambda t: (k * np.exp(-lam * t) if t < limit else 0)

def probability(p):
    """Return true with probability p."""
    return p > np.random.uniform(0.0, 1.0)

def _simulated_annealing(problem, schedule=exp_schedule()):
    """[Figure 4.5] CAUTION: This differs from the pseudocode as it
    returns a state instead of a Node."""
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = np.random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(np.exp(delta_e / T)):
            current = next_choice


In [12]:
w = np.array([25, 46, 29], dtype=float)
r = np.array([15, 10, 2], dtype=float)
p = Portfolio(initial=w, restitution=r, step=0.1)
res = _simulated_annealing(p)
print(res)
p.value(res)

[27. 44. 29.]


352.832146825476